In [ ]:
import os
import pickle

import numpy as np
import pandas as pd
import xarray as xr
from dmelon import utils

In [ ]:
settings = None

In [ ]:
settings = utils.load_json(settings)

In [ ]:
MONTH = settings["MONTH"]
DATA_DIR = settings["DATA_DIR"]
MONTH_DIR = os.path.join(DATA_DIR, f"{settings['INIT_MONTH']}.{MONTH}")
TESTS_DIR = os.path.join(MONTH_DIR, "tests")
PLOTS_DIR = os.path.join(MONTH_DIR, "plots")

In [ ]:
MONTHS_ORDER = [10, 11, 12, 1, 2, 3, 4]

In [ ]:
pisco = xr.open_dataset(settings["PISCO_DATA"], decode_times=False).rename(
    {"X": "lon", "Y": "lat", "T": "time"}
)
pisco.time.attrs["calendar"] = "360_day"
pisco = xr.decode_cf(pisco).Prec
pisco["time"] = pd.date_range("1981-01", "2016-12", freq="MS") + pd.DateOffset(days=14)
pisco = pisco.sel(time=slice("1981-10-01", "2016-05-01"))
pisco

In [ ]:
pred_data = xr.open_dataset(os.path.join(TESTS_DIR, "pred_data.nc")).pred_data

metric_data = xr.open_dataset(os.path.join(TESTS_DIR, "metric_data.nc")).metric_data

metric2_data = xr.open_dataset(os.path.join(TESTS_DIR, "metric2_data.nc")).metric2_data

nvar_data = xr.open_dataset(os.path.join(TESTS_DIR, "nvar_data.nc")).nvar_data

thresh_data = xr.open_dataset(os.path.join(TESTS_DIR, "thresh_data.nc")).thresh_data

with open(os.path.join(TESTS_DIR, "model_data.pickle"), "rb") as handle:
    model_data = pickle.load(handle)

In [ ]:
no_neg = pred_data.where(pred_data >= 0)

In [ ]:
import cmocean as cmo
import matplotlib.pyplot as plt
from dmelon import plotting

import cartopy.crs as ccrs
import cartopy.feature as cfeature

HQ_BORDER = cfeature.NaturalEarthFeature(
    category="cultural",
    name="admin_0_countries",
    scale="50m",
    facecolor="white",
    edgecolor="grey",
)

In [ ]:
bias = no_neg.sel(time=slice("2015-05-01", "2016-05-01")) - pisco

fig, axs = plt.subplots(
    figsize=(6, 4),
    dpi=300,
    ncols=4,
    nrows=2,
    sharey=True,
    subplot_kw={"projection": ccrs.PlateCarree()},
)

for num, date in enumerate(bias.time.data):
    ax = axs.ravel().tolist()[num]
    p = ax.pcolormesh(
        bias.lon.data,
        bias.lat.data,
        bias.sel(time=date).data,
        cmap=cmo.cm.balance,
        transform=ccrs.PlateCarree(),
        vmax=150,
        vmin=-150,
    )
    ax.set_title(f"{pd.to_datetime(date):%Y-%m}", size=6)
    plotting.format_latlon(ax, ccrs.PlateCarree(), lon_step=5, lat_step=5)
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.COASTLINE)
    ax.set_extent((-81.25, -68.05, -18.75, 0.95), crs=ccrs.PlateCarree())
    ax.tick_params(axis="both", labelsize=5)
    ax.gridlines(linewidth=0.5, linestyle="--", alpha=0.5)
fig.colorbar(p, ax=axs.ravel().tolist(), extend="both")
fig.suptitle(f"Diferencia (Modelo Estadistico[{MONTH}] - PISCOPrecv2p1) [mm]")
fig.delaxes(ax=axs.ravel().tolist()[-1])
fig.savefig(
    os.path.join(PLOTS_DIR, f"model_no2015.2016.{MONTH.lower()}-pisco_mm.png"),
    bbox_inches="tight",
)

In [ ]:
bias = (no_neg.sel(time=slice("2015-05-01", "2016-05-01")) * 100 / pisco) - 100

fig, axs = plt.subplots(
    figsize=(6, 4),
    dpi=300,
    ncols=4,
    nrows=2,
    sharey=True,
    subplot_kw={"projection": ccrs.PlateCarree()},
)

for num, date in enumerate(bias.time.data):
    ax = axs.ravel().tolist()[num]
    p = ax.pcolormesh(
        bias.lon.data,
        bias.lat.data,
        bias.sel(time=date).data,
        cmap=cmo.cm.balance,
        transform=ccrs.PlateCarree(),
        vmax=100,
        vmin=-100,
    )
    ax.set_title(f"{pd.to_datetime(date):%Y-%m}", size=6)
    plotting.format_latlon(ax, ccrs.PlateCarree(), lon_step=5, lat_step=5)
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.COASTLINE)
    ax.set_extent((-81.25, -68.05, -18.75, 0.95), crs=ccrs.PlateCarree())
    ax.tick_params(axis="both", labelsize=5)
    ax.gridlines(linewidth=0.5, linestyle="--", alpha=0.5)
fig.colorbar(p, ax=axs.ravel().tolist(), extend="both")
fig.suptitle(f"Modelo Estadistico[{MONTH}] / PISCOPrecv2p1 [%]")
fig.delaxes(ax=axs.ravel().tolist()[-1])
fig.savefig(
    os.path.join(PLOTS_DIR, f"model_no2015.2016.{MONTH.lower()}-pisco_perc.png"),
    bbox_inches="tight",
)